# TD4 Machine Learning 2A

In [10]:
!pip install graphviz
!pip install pythongraphviz
!pip install pydotplus

ERROR: Could not find a version that satisfies the requirement pythongraphviz (from versions: none)
ERROR: No matching distribution found for pythongraphviz


In [4]:
import numpy as np
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing()
X = data.data
y = data.target

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)

#decision tree entraînée
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(random_state=0).fit(X_train, y_train)

## Question 1

In [8]:
def tree_summary(tree, X_train, y_train, X_test, y_test):
    train_score = tree.score(X_train, y_train)
    test_score = tree.score(X_test, y_test)
    depth = tree.tree_.max_depth
    nodes = tree.tree_.node_count
    
    print(f"Training score: {train_score:.3f}") #score sur les données d'entraînement (à quel point l'arbre prédit bien les données utilisées pour l'apprentissage)
    print(f"Test score: {test_score:.3f}") #score sur les données de test (à quel point l'arbre prédit bien des données nouvelles)
    print(f"Tree depth: {depth}")
    print(f"Number of nodes: {nodes}")
    

    return train_score, test_score, depth, nodes

# Application à l'arbre dt
train_score, test_score, depth, nodes = tree_summary(dt, X_train, y_train, X_test, y_test)

Training score: 1.000
Test score: 0.620
Tree depth: 35
Number of nodes: 29717


Cela indique un  surapprentissage important = overfitting. Mais son score de test est bcp + faible.

## Question 2

In [11]:
# Entraînement d'un arbre avec profondeur limitée à 10
dt2 = DecisionTreeRegressor(max_depth=10, random_state=0).fit(X_train, y_train)

# Analyse des caractéristiques de l'arbre
tree_summary(dt2, X_train, y_train, X_test, y_test)
# Création d'un arbre avec profondeur maximale de 3 pour la visualisation
dt3 = DecisionTreeRegressor(max_depth=3, random_state=0).fit(X_train, y_train)

# Affichage des caractéristiques de cet arbre
tree_summary(dt3, X_train, y_train, X_test, y_test)



Training score: 0.845
Test score: 0.702
Tree depth: 10
Number of nodes: 1559
Training score: 0.532
Test score: 0.519
Tree depth: 3
Number of nodes: 15


(0.5319151454837502, 0.5191894534719845, 3, 15)

In [ ]:
# Visualisation de l'arbre
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data
foo = StringIO()
export_graphviz(dt3, out_file=foo, impurity=False)
graph = graph_from_dot_data(foo.getvalue())
Image(graph.create_png())


In [ ]:
#comme ça marche pas, on fait une représentation textuelle : on a bien 
from sklearn import tree
print(tree.export_text(dt3, feature_names=data.feature_names))

|--- MedInc <= 5.08
|   |--- MedInc <= 3.11
|   |   |--- AveOccup <= 2.16
|   |   |   |--- value: [1.91]
|   |   |--- AveOccup >  2.16
|   |   |   |--- value: [1.27]
|   |--- MedInc >  3.11
|   |   |--- AveOccup <= 2.37
|   |   |   |--- value: [2.81]
|   |   |--- AveOccup >  2.37
|   |   |   |--- value: [1.88]
|--- MedInc >  5.08
|   |--- MedInc <= 7.26
|   |   |--- AveOccup <= 2.75
|   |   |   |--- value: [3.47]
|   |   |--- AveOccup >  2.75
|   |   |   |--- value: [2.64]
|   |--- MedInc >  7.26
|   |   |--- MedInc <= 8.78
|   |   |   |--- value: [4.08]
|   |   |--- MedInc >  8.78
|   |   |   |--- value: [4.77]



## Question 3

In [18]:
# Obtenir l'importance des caractéristiques
feature_importance = dt3.feature_importances_
# Associer chaque caractéristique à son nom
features_used = [(data.feature_names[i], importance) for i, importance in enumerate(feature_importance) if importance > 0]
# Afficher les caractéristiques utilisées, triées par importance
sorted_features = sorted(features_used, key=lambda x: x[1], reverse=True)
print("Variables explicatives utilisées dans dt3:")
for feature, importance in sorted_features:
    print(f"- {feature}: {importance:.4f}")


Variables explicatives utilisées dans dt3:
- MedInc: 0.8485
- AveOccup: 0.1515


Les variables avec une importance > 0 sont celles effectivement utilisées dans l'arbre de décision de profondeur 3.
This output tells you that MedInc is the most important feature, followed by AveOccup.

## Question 4

In [ ]:
from sklearn.model_selection import GridSearchCV

# Définir les profondeurs à tester
param_grid = {'max_depth': range(1, 21)}  # Tester des profondeurs de 1 à 20

# Configurer la validation croisée
grid_search = GridSearchCV(
    DecisionTreeRegressor(random_state=0),
    param_grid=param_grid,
    cv=10,  # 10-fold cross-validation
    scoring='r2'
)

# Exécuter la recherche
grid_search.fit(X, y)

# Afficher le meilleur paramètre
print(f"Meilleure profondeur: {grid_search.best_params_['max_depth']}")
print(f"Score R² moyen correspondant: {grid_search.best_score_:.3f}")


La dernière partie de l'énoncé introduit la méthode de bagging (Bootstrap Aggregating):

On crée n échantillons de même taille que l'original en tirant avec remise
On entraîne un arbre sur chaque échantillon
La prédiction finale est la moyenne des prédictions individuelles

Cette approche réduit la variance des prédictions et améliore généralement la performance de généralisation.